In [ ]:
# !apt-get update && apt-get install swig cmake
# !pip install box2d-py
# !pip install "stable-baselines3[extra]>=2.0.0a4"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# import gymnasium as gym
import gym
# from gymnasium import Env, spaces
from gym import Env, spaces
import pandas as pd
import random
import os

import torch
from torch import nn

from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
PATH = "drive/MyDrive/Trading_bot/"

In [ ]:
INITIAL_BALANCE = 10000

class TradingEnv(Env):
  def __init__(self, data_path, initial_step=0, end_step=0, random_steps=False, forward=True):
    #self.reset() maybe
    self.df = self.__read_df(data_path)
    self.action_space = spaces.Discrete(3)
    # self.action_space = spaces.Tuple((spaces.Discrete(3),
    #                                  spaces.Box(low=0, high=1, shape=(1,)))) # Buy, Sell, do Nothing
    self.observation_space = spaces.Box(low=0, high=100000, shape=(6, 6), dtype=np.float32)
    self.current_step = initial_step
    self.end_step = self.__clamp(end_step)
    self.random_steps = random_steps
    self.forward = forward



  def __read_df(self, data_path):
    df = pd.read_csv(data_path, quotechar='"')
    return df

  def reset(self):
    self.shares_held = 0
    self.balance = INITIAL_BALANCE
    self.current_step = 0
    if self.random_steps:
      self.current_step = random.randint(0, self.df.shape[0] - 6)
      self.end_step = random.randint(self.current_step, len(self.df.loc[:, 'Open'].values - 6))
    self.net_worth = INITIAL_BALANCE
    self.shares_sold = 0
    self.total_shares_sold_value = 0.0
    self.last_price = random.uniform(
          self.df.loc[self.current_step, 'Open'],
          self.df.loc[self.current_step, 'Close']
      )
    return self.__getNextObs()

  def render(self):
    # print(f"balance : {self.balance}, portfolio : {self.shares_held * self.last_price}, net_worth : {self.net_worth}")
    print(f"net_worth : {self.net_worth}")

  def step(self, action):
    prev_net_worth = self.net_worth
    self.__take_action(action)
    self.current_step += 1
    if self.current_step > len(self.df.loc[:, 'Open'].values) - 6:
        self.current_step = 0
    returns = (self.net_worth - prev_net_worth) / prev_net_worth
    reward = self.__compute_reward(action, prev_net_worth) # TODO : change to action[0] after stable baselines
    done = self.net_worth <= 0 or self.current_step == self.end_step
    obs = self.__getNextObs()
    return obs, reward, done, {}

  def __getNextObs(self):
    frame = np.array([
        self.df.iloc[self.current_step: self.current_step + 6, self.df.columns.get_loc('Open')],
        self.df.iloc[self.current_step: self.current_step + 6, self.df.columns.get_loc('High')],
        self.df.iloc[self.current_step: self.current_step + 6, self.df.columns.get_loc('Low')],
        self.df.iloc[self.current_step: self.current_step + 6, self.df.columns.get_loc('Close')],
        self.df.iloc[self.current_step: self.current_step + 6, self.df.columns.get_loc('Volume')]
    ])
    obs = np.append(frame, [[
          self.balance,
          self.net_worth,
          self.shares_held,
          self.shares_sold,
          self.last_price,
          self.total_shares_sold_value
      ]], axis=0)
    return obs

  def __compute_reward(self, action, prev_net_worth):
    returns = (self.net_worth - prev_net_worth) / prev_net_worth
    reward = 0.0
    if action != 2:
      #penalize the bot for making a trade
      reward -= .0002
    reward += returns
    if returns == 0.0:
      reward -= .0005
    return reward
  def __take_action(self, action):
    #select a price 
    current_price = random.uniform(
          self.df.loc[self.current_step, 'Open'],
          self.df.loc[self.current_step, 'Close']
      )
    self.last_price = current_price
    # here amount refers to the percentage of the account balance
    amount = .2
    if action == 0:

      quantity = amount * self.balance / current_price
      self.shares_held += quantity
      self.balance -= amount * self.balance
    elif action == 1:
      #Sell
      quantity = amount * self.shares_held
      self.shares_held -= quantity
      self.shares_sold += quantity
      self.total_shares_sold_value += quantity * current_price
      self.balance += quantity * current_price
    self.net_worth = self.balance + self.shares_held * current_price
  def __clamp(self, end_step):
    if 0 < end_step < self.df.shape[0] - 5:
      return end_step
    return self.df.shape[0] - 6

In [ ]:
TSLA_path = os.path.join(PATH, "data","TSLA.csv")
MSFT_path = os.path.join(PATH, "data", "MSFT.csv")
AAPL_path = os.path.join(PATH, "data", "AAPL.csv")

In [ ]:
#creating the training environment of different stocks

TSLA_training_env = TradingEnv(TSLA_path, initial_step=0, end_step=837)
MSFT_training_env = TradingEnv(MSFT_path, initial_step=0, end_step=837)
AAPL_training_env = TradingEnv(TSLA_path, initial_step=0, end_step=837)

In [ ]:
print(TSLA_training_env.reset())

[[2.10000000e+02 2.17009995e+02 2.25949997e+02 2.52699997e+02
  2.50100006e+02 2.50050003e+02]
 [2.16979996e+02 2.18860001e+02 2.30610001e+02 2.54630005e+02
  2.57179993e+02 2.55399994e+02]
 [2.02509995e+02 2.10419998e+02 2.16779999e+02 2.42080002e+02
  2.47080002e+02 2.48479996e+02]
 [2.12080002e+02 2.11990005e+02 2.20110001e+02 2.42679993e+02
  2.54850006e+02 2.53919998e+02]
 [1.50327800e+08 1.37734000e+08 1.70772700e+08 1.25147800e+08
  9.35629000e+07 8.89172000e+07]
 [1.00000000e+04 1.00000000e+04 0.00000000e+00 0.00000000e+00
  1.94735289e+01 0.00000000e+00]]


In [ ]:
model = A2C("MlpPolicy", MSFT_training_env, verbose=1)
model.learn(total_timesteps=5000)

In [ ]:
#save the model
model.save("Trading_bot_MSFT_5000")

In [ ]:
del model